In [28]:
import sys
sys.path.append('../')
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.signal import convolve2d
from scipy.optimize import curve_fit
from scipy.optimize import brute
# import cv2
import torch
import json
import csv
from models import *
from functions.all_knots_functions import *
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from sklearn.model_selection import train_test_split
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import collections
import itertools
import torch

import torch.nn.functional as F
from tqdm import trange
import pickle
from sklearn.metrics import confusion_matrix
import seaborn as sns

from torchsummary import summary
device = torch.device("cpu")
# from torchinfo import summary

In [29]:

def print_layer_info(model):
    for name, module in model.named_modules():
        # Check for conv layers (2D or 3D)
        if isinstance(module, (nn.Conv2d, nn.Conv3d)):
            print(f"Layer: {name}")
            print(f"  Type: {module.__class__.__name__}")
            print(f"  In Channels: {module.in_channels}, Out Channels: {module.out_channels}")
            print(f"  Kernel Size: {module.kernel_size}")
            print(f"  Stride: {module.stride}")
            print(f"  Padding: {module.padding}")
            print(f"  Dilation: {module.dilation}")
            print()
        # Check for linear layers
        elif isinstance(module, nn.Linear):
            print(f"Layer: {name}")
            print(f"  Type: {module.__class__.__name__}")
            print(f"  In Features: {module.in_features}, Out Features: {module.out_features}")
            print()

In [30]:

# Load the saved model
model_path = "classifier_4foil_3d_full_2.pth"
checkpoint = torch.load(model_path)

# Extract the saved structure and hyperparameters
stages = checkpoint['stages']
pooling_configs = checkpoint['pooling_configs']
num_classes = checkpoint['num_classes']
desired_res = checkpoint['desired_res']

# Initialize the model and load the saved state dict
model_3D = Classifier3D(stages, pooling_configs, num_classes=num_classes).to(device)
model_3D.load_state_dict(checkpoint['model_state_dict'])
model_3D.eval()  # Set the model to evaluation mode

# print("Model loaded and ready for inference!")
print('stages: \n', stages)
print('pooling_configs: \n',pooling_configs)
print('num_classes: \n', num_classes)
summary(model_3D, input_size=(1, 32, 32, 32), device='cpu')

stages: 
 [[(1, 32, 3, 1, 1), (32, 32, 3, 1, 1), (32, 32, 3, 1, 1)], [(32, 64, 5, 1, 1), (64, 64, 5, 1, 1), (64, 64, 5, 1, 1)]]
pooling_configs: 
 [(2, 2, 1), (2, 2, 1)]
num_classes: 
 81
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 32, 32, 32]             896
       BatchNorm3d-2       [-1, 32, 32, 32, 32]              64
              ReLU-3       [-1, 32, 32, 32, 32]               0
            Conv3d-4       [-1, 32, 32, 32, 32]          27,680
       BatchNorm3d-5       [-1, 32, 32, 32, 32]              64
              ReLU-6       [-1, 32, 32, 32, 32]               0
            Conv3d-7       [-1, 32, 32, 32, 32]          27,680
       BatchNorm3d-8       [-1, 32, 32, 32, 32]              64
              ReLU-9       [-1, 32, 32, 32, 32]               0
        MaxPool3d-10       [-1, 32, 17, 17, 17]               0
           Conv3d-11       [-1, 64, 15, 15,

In [31]:
summary(model_3D, input_size=(1, 1, 32, 32, 32), verbose=2)


TypeError: summary() got an unexpected keyword argument 'verbose'

In [ ]:
import torch.onnx
dummy_input = torch.randn(1, 1, 32, 32, 32).to('cpu')
torch.onnx.export(model_3D.to('cpu'), dummy_input, "my_model.onnx", opset_version=11)

In [ ]:
hyperparams = {
    'learning_rate': 1e-5,  # Control dropout rate
    'patience': 0,  # Number of epochs between learning rate decay
    'decay_epoch': 25,
    'factor': 0.2,  # Multiplicative factor of learning rate decay
    'batch_size': 64
}
num_epochs = 50


In [32]:

# Load the saved model
model_path = "classifier_knots_3d_32_full_3.pth"
checkpoint = torch.load(model_path)

# Extract the saved structure and hyperparameters
stages = checkpoint['stages']
pooling_configs = checkpoint['pooling_configs']
num_classes = checkpoint['num_classes']
# desired_res = checkpoint['desired_res']

# Initialize the model and load the saved state dict
model_3D_H = Classifier3D(stages, pooling_configs, num_classes=num_classes).to(device)
model_3D_H.load_state_dict(checkpoint['model_state_dict'])
model_3D_H.eval()  # Set the model to evaluation mode

# print("Model loaded and ready for inference!")
# print(stages, pooling_configs, num_classes)

summary(model_3D_H, input_size=(1, 32, 32, 32), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1       [-1, 32, 32, 32, 32]             896
       BatchNorm3d-2       [-1, 32, 32, 32, 32]              64
              ReLU-3       [-1, 32, 32, 32, 32]               0
            Conv3d-4       [-1, 32, 32, 32, 32]          27,680
       BatchNorm3d-5       [-1, 32, 32, 32, 32]              64
              ReLU-6       [-1, 32, 32, 32, 32]               0
            Conv3d-7       [-1, 32, 32, 32, 32]          27,680
       BatchNorm3d-8       [-1, 32, 32, 32, 32]              64
              ReLU-9       [-1, 32, 32, 32, 32]               0
        MaxPool3d-10       [-1, 32, 17, 17, 17]               0
           Conv3d-11       [-1, 64, 15, 15, 15]         256,064
      BatchNorm3d-12       [-1, 64, 15, 15, 15]             128
             ReLU-13       [-1, 64, 15, 15, 15]               0
           Conv3d-14       [-1, 64, 13,

In [33]:
input_size = 91
# Load the saved model
model_path = "classifier_new_spec10_full_3.pth"
checkpoint = torch.load(model_path)

# Extract the saved structure and hyperparameters
hidden_sizes1 = checkpoint['hidden_sizes1']
hidden_sizes2 = checkpoint['hidden_sizes2']
hidden_sizes3 = checkpoint['hidden_sizes3']
num_classes = checkpoint['num_classes']
num_hidden = checkpoint['num_hidden']

# Initialize the model and load the saved state dict
model_FC = ClassifierFC_spec(input_size, hidden_sizes1, hidden_sizes2, hidden_sizes3, num_hidden, num_classes=num_classes).to(device)
model_FC.load_state_dict(checkpoint['model_state_dict'])
model_FC.eval()  # Set the model to evaluation mode

# print("Model loaded and ready for inference!")
summary(model_FC, input_size=(91, ), device='cpu')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]          11,776
       BatchNorm1d-2                  [-1, 128]             256
              ReLU-3                  [-1, 128]               0
           Dropout-4                  [-1, 128]               0
            Linear-5                 [-1, 1024]         132,096
       BatchNorm1d-6                 [-1, 1024]           2,048
              ReLU-7                 [-1, 1024]               0
           Dropout-8                 [-1, 1024]               0
            Linear-9                 [-1, 1024]       1,049,600
      BatchNorm1d-10                 [-1, 1024]           2,048
             ReLU-11                 [-1, 1024]               0
          Dropout-12                 [-1, 1024]               0
           Linear-13                 [-1, 1024]       1,049,600
      BatchNorm1d-14                 [-

In [36]:
# Load the saved model
model_path = "classifier_spec10_conv_3.pth"
checkpoint = torch.load(model_path)

# Extract the saved structure and hyperparameters
stages = checkpoint['stages']
pooling_configs = checkpoint['pooling_configs']
num_classes = checkpoint['num_classes']
print(stages, pooling_configs)

# Initialize the model and load the saved state dict
model_2D = Classifier2D(stages, pooling_configs, num_classes=num_classes).to(device)
model_2D.load_state_dict(checkpoint['model_state_dict'])
model_2D.eval()  # Set the model to evaluation mode
# print("Model loaded and ready for inference!")
summary(model_2D, input_size=(91, ), device='cpu')

[[(1, 32, 3, 1, 1), (32, 32, 3, 1, 1), (32, 32, 3, 1, 1)], [(32, 64, 3, 1, 1), (64, 64, 3, 1, 1), (64, 64, 3, 1, 1)]] [(2, 2, 1), (2, 2, 1)]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 32, 7, 13]             320
       BatchNorm2d-2            [-1, 32, 7, 13]              64
              ReLU-3            [-1, 32, 7, 13]               0
            Conv2d-4            [-1, 32, 7, 13]           9,248
       BatchNorm2d-5            [-1, 32, 7, 13]              64
              ReLU-6            [-1, 32, 7, 13]               0
            Conv2d-7            [-1, 32, 7, 13]           9,248
       BatchNorm2d-8            [-1, 32, 7, 13]              64
              ReLU-9            [-1, 32, 7, 13]               0
        MaxPool2d-10             [-1, 32, 4, 7]               0
           Conv2d-11             [-1, 64, 4, 7]          18,496
      BatchNorm2d-12      